# Hi kagglers 🙋🏻‍♂️ and Welcome to this new competition!

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import random
import gc
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve,RocCurveDisplay,ConfusionMatrixDisplay,confusion_matrix,roc_auc_score,accuracy_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')
y = train['target']

# Some Exploratory Data Analysis

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
#Check if there'is null values
train.isnull().sum()

In [ ]:
#Check if there'is null values
test.isnull().sum()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
cols = ['f'+str(i) for i in range(100)]

In [ ]:
# plot the first 32 features 
i = 1
plt.figure()
fig, ax = plt.subplots(8, 4,figsize=(20, 22))
for feature in cols[:32]:
    plt.subplot(8, 4,i)
    sns.histplot(train[feature],color="blue", kde=True,bins=100, label='train_'+feature)
    sns.histplot(test[feature],color="olive", kde=True,bins=100, label='test_'+feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
sns.catplot(x="target", kind="count", palette="ch:.25", data=train)

In [ ]:
## Target distibution
pie, ax = plt.subplots(figsize=[18,8])
train.groupby('target').size().plot(kind='pie',autopct='%.1f',ax=ax,title='Target distibution')

# Modeling

In [ ]:
# apply standard scaler to the data
scaler = StandardScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

In [ ]:
preds = np.zeros(test.shape[0])
kf = StratifiedKFold(n_splits=15,random_state=48,shuffle=True)
auc=[]  # list contains auc for each fold
acc=[]  # list contains accuracy for each fold
n=0
for trn_idx, test_idx in kf.split(train[cols],y):
    X_tr,X_val=train[cols].iloc[trn_idx],train[cols].iloc[test_idx]
    y_tr,y_val=y.iloc[trn_idx],y.iloc[test_idx]
    
    model = LogisticRegression(solver='liblinear')
    model.fit(X_tr,y_tr)
    preds += model.predict_proba(test[cols])[:,1]/kf.n_splits
    
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]))
    acc.append(accuracy_score(y_val, model.predict(X_val)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    n+=1  

In [ ]:
print(f"the mean AUC is : {round(np.mean(auc)*100,2)} while the mean Accuracy is : {round(np.mean(acc)*100,2)} ")

* Binary classification problem has only two classes to classify like in this competition, preferably a positive and a negative class.
* <b> True Positive (TP)</b>: It refers to the number of predictions where the classifier correctly predicts the positive class as positive.
* <b>True Negative (TN)</b>: It refers to the number of predictions where the classifier correctly predicts the negative class as negative.
* <b>False Positive (FP)</b>: It refers to the number of predictions where the classifier incorrectly predicts the negative class as positive.
* <b>False Negative (FN)</b>: It refers to the number of predictions where the classifier incorrectly predicts the positive class as negative.
<img src="https://miro.medium.com/max/1000/1*fxiTNIgOyvAombPJx5KGeA.png" width="600px"/>

In [ ]:
# Plot of confusion matrix for the last fold
cm = confusion_matrix(model.predict(X_val),y_val)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Plot of roc curve for the last fold
y_pred_proba = model.predict_proba(X_val)[:, 1]
fpr, tpr, _ = metrics.roc_curve(y_val,  y_pred_proba)
auc = metrics.roc_auc_score(y_val, y_pred_proba)
plt.plot(fpr,tpr,label="data, auc for fold 15="+str(round(auc*100,2)))
plt.legend(loc=4)
plt.show()

# Let's Make a Submission

In [ ]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)

In [ ]:
sub

## I hope that you find this kernel usefull🏄